In [32]:
# 먼저 전체 데이터 개수 확인
import requests
import json
import pandas as pd
from time import sleep

API_KEY = "8434c606f68343abaeb9"
SERVICE_ID = "COOKRCP01"
DATA_TYPE = "json"

# 첫 번째 요청으로 전체 데이터 개수 확인
url = f"http://openapi.foodsafetykorea.go.kr/api/{API_KEY}/{SERVICE_ID}/{DATA_TYPE}/1/1"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    if 'COOKRCP01' in data:
        total_count = data['COOKRCP01']['total_count']
        print(f"전체 데이터 개수: {total_count}개")
    else:
        print("데이터 구조 확인:")
        print(data)
else:
    print(f"API 요청 실패: {response.status_code}")
    print(response.text)

전체 데이터 개수: 1136개


In [33]:
def fetch_all_recipe_data():
    """
    모든 레시피 데이터를 1000개씩 호출하여 가져오는 함수
    """
    API_KEY = "8434c606f68343abaeb9"
    SERVICE_ID = "COOKRCP01"
    DATA_TYPE = "json"
    BATCH_SIZE = 1000
    
    all_recipes = []
    start_idx = 1
    
    while True:
        end_idx = start_idx + BATCH_SIZE - 1
        url = f"http://openapi.foodsafetykorea.go.kr/api/{API_KEY}/{SERVICE_ID}/{DATA_TYPE}/{start_idx}/{end_idx}"
        
        print(f"요청 중: {start_idx} ~ {end_idx}")
        
        try:
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                
                if 'COOKRCP01' in data:
                    # 첫 번째 호출에서 전체 개수 확인
                    if start_idx == 1:
                        total_count = data['COOKRCP01']['total_count']
                        print(f"전체 데이터 개수: {total_count}개")
                    
                    # 데이터가 있는지 확인
                    if 'row' in data['COOKRCP01']:
                        recipes = data['COOKRCP01']['row']
                        all_recipes.extend(recipes)
                        print(f"현재까지 수집된 데이터: {len(all_recipes)}개")
                        
                        # 받은 데이터가 BATCH_SIZE보다 적으면 마지막 배치
                        if len(recipes) < BATCH_SIZE:
                            print("마지막 배치 완료")
                            break
                    else:
                        print("더 이상 데이터가 없습니다.")
                        break
                else:
                    print("예상하지 못한 응답 구조:", data)
                    break
            else:
                print(f"API 요청 실패: {response.status_code}")
                break
                
        except Exception as e:
            print(f"오류 발생: {e}")
            break
        
        start_idx += BATCH_SIZE
        
        # API 호출 간 잠시 대기 (서버 부하 방지)
        sleep(0.1)
    
    print(f"\\n총 {len(all_recipes)}개의 레시피 데이터를 수집했습니다.")
    return all_recipes

# 모든 데이터 수집 실행
print("레시피 데이터 수집을 시작합니다...")
recipes_data = fetch_all_recipe_data()

레시피 데이터 수집을 시작합니다...
요청 중: 1 ~ 1000
전체 데이터 개수: 1136개
현재까지 수집된 데이터: 1000개
요청 중: 1001 ~ 2000
현재까지 수집된 데이터: 1136개
마지막 배치 완료
\n총 1136개의 레시피 데이터를 수집했습니다.


In [34]:
# 수집한 데이터를 DataFrame으로 변환
if recipes_data:
    df = pd.DataFrame(recipes_data)
    
    print(f"DataFrame 생성 완료!")
    print(f"총 레코드 수: {len(df)}")
    print(f"컬럼 수: {len(df.columns)}")
    print(f"DataFrame 크기: {df.shape}")
    print(f"\\n컬럼 목록:")
    print(list(df.columns))
    
    # 주요 컬럼들 미리보기
    key_columns = ['RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_ENG']
    available_columns = [col for col in key_columns if col in df.columns]
    
    if available_columns:
        print(f"\\n주요 데이터 미리보기 (처음 5개):")
        print(df[available_columns].head())
    
    # 데이터 타입 확인
    print(f"\\n데이터 타입:")
    print(df.dtypes)
    
    # 결측값 확인
    print(f"\\n결측값 개수:")
    print(df.isnull().sum())
    
else:
    print("수집된 데이터가 없습니다.")
    df = pd.DataFrame()

DataFrame 생성 완료!
총 레코드 수: 1136
컬럼 수: 55
DataFrame 크기: (1136, 55)
\n컬럼 목록:
['RCP_PARTS_DTLS', 'RCP_WAY2', 'MANUAL_IMG20', 'MANUAL20', 'RCP_SEQ', 'INFO_NA', 'INFO_WGT', 'INFO_PRO', 'MANUAL_IMG13', 'MANUAL_IMG14', 'MANUAL_IMG15', 'MANUAL_IMG16', 'MANUAL_IMG10', 'MANUAL_IMG11', 'MANUAL_IMG12', 'MANUAL_IMG17', 'MANUAL_IMG18', 'MANUAL_IMG19', 'INFO_FAT', 'HASH_TAG', 'MANUAL_IMG02', 'MANUAL_IMG03', 'RCP_PAT2', 'MANUAL_IMG04', 'MANUAL_IMG05', 'MANUAL_IMG01', 'MANUAL01', 'ATT_FILE_NO_MK', 'MANUAL_IMG06', 'MANUAL_IMG07', 'MANUAL_IMG08', 'MANUAL_IMG09', 'MANUAL08', 'MANUAL09', 'MANUAL06', 'MANUAL07', 'MANUAL04', 'MANUAL05', 'MANUAL02', 'MANUAL03', 'ATT_FILE_NO_MAIN', 'MANUAL11', 'MANUAL12', 'MANUAL10', 'INFO_CAR', 'MANUAL19', 'RCP_NA_TIP', 'INFO_ENG', 'MANUAL17', 'MANUAL18', 'RCP_NM', 'MANUAL15', 'MANUAL16', 'MANUAL13', 'MANUAL14']
\n주요 데이터 미리보기 (처음 5개):
           RCP_NM RCP_WAY2 RCP_PAT2 INFO_ENG
0       새우 두부 계란찜       찌기       반찬      220
1        부추 콩가루 찜       찌기       반찬      215
2       방

In [35]:
# 현재 DataFrame의 컬럼 구조 분석
if 'df' in locals() and not df.empty:
    print("현재 DataFrame 컬럼들:")
    print("=" * 50)
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")
    
    print(f"\n총 컬럼 수: {len(df.columns)}")
    
    # 샘플 데이터로 컬럼 내용 확인
    print("\n첫 번째 레시피의 컬럼별 데이터:")
    print("=" * 50)
    if len(df) > 0:
        for col in df.columns:
            value = df.iloc[0][col]
            if isinstance(value, str) and len(value) > 100:
                print(f"{col}: {value[:100]}...")
            else:
                print(f"{col}: {value}")
else:
    print("DataFrame이 생성되지 않았습니다. 먼저 데이터를 수집해주세요.")

현재 DataFrame 컬럼들:
 1. RCP_PARTS_DTLS
 2. RCP_WAY2
 3. MANUAL_IMG20
 4. MANUAL20
 5. RCP_SEQ
 6. INFO_NA
 7. INFO_WGT
 8. INFO_PRO
 9. MANUAL_IMG13
10. MANUAL_IMG14
11. MANUAL_IMG15
12. MANUAL_IMG16
13. MANUAL_IMG10
14. MANUAL_IMG11
15. MANUAL_IMG12
16. MANUAL_IMG17
17. MANUAL_IMG18
18. MANUAL_IMG19
19. INFO_FAT
20. HASH_TAG
21. MANUAL_IMG02
22. MANUAL_IMG03
23. RCP_PAT2
24. MANUAL_IMG04
25. MANUAL_IMG05
26. MANUAL_IMG01
27. MANUAL01
28. ATT_FILE_NO_MK
29. MANUAL_IMG06
30. MANUAL_IMG07
31. MANUAL_IMG08
32. MANUAL_IMG09
33. MANUAL08
34. MANUAL09
35. MANUAL06
36. MANUAL07
37. MANUAL04
38. MANUAL05
39. MANUAL02
40. MANUAL03
41. ATT_FILE_NO_MAIN
42. MANUAL11
43. MANUAL12
44. MANUAL10
45. INFO_CAR
46. MANUAL19
47. RCP_NA_TIP
48. INFO_ENG
49. MANUAL17
50. MANUAL18
51. RCP_NM
52. MANUAL15
53. MANUAL16
54. MANUAL13
55. MANUAL14

총 컬럼 수: 55

첫 번째 레시피의 컬럼별 데이터:
RCP_PARTS_DTLS: 새우두부계란찜
연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)
고명
시금치...
RCP_WAY2: 찌기
MANU

In [36]:
# 컬럼들을 논리적 순서로 정렬 (수정된 버전)
def reorder_recipe_columns(df):
    """
    레시피 DataFrame의 컬럼을 논리적 순서로 정렬
    """
    if df.empty:
        return df
    
    # 기본 정보 컬럼들
    basic_info = ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA']
    
    # 재료 정보 컬럼들  
    ingredient_info = ['RCP_PARTS_DTLS']
    
    # 레시피 제작 순서 컬럼들 (MANUAL01~MANUAL20)
    manual_columns = []
    for i in range(1, 21):
        manual_col = f'MANUAL{i:02d}'
        if manual_col in df.columns:
            manual_columns.append(manual_col)
    
    # 이미지 정보 컬럼들 (MANUAL_IMG01~MANUAL_IMG20)
    image_columns = []
    for i in range(1, 21):
        img_col = f'MANUAL_IMG{i:02d}'
        if img_col in df.columns:
            image_columns.append(img_col)
    
    # 메인 이미지 컬럼들
    main_image = ['ATT_FILE_NO_MAIN', 'ATT_FILE_NO_MK']
    
    # 기타 컬럼들 (위에서 분류되지 않은 것들)
    all_categorized = set(basic_info + ingredient_info + manual_columns + image_columns + main_image)
    other_columns = [col for col in df.columns if col not in all_categorized]
    
    # 최종 컬럼 순서 결정
    ordered_columns = []
    
    # 1. 기본 정보
    ordered_columns.extend([col for col in basic_info if col in df.columns])
    
    # 2. 재료 정보
    ordered_columns.extend([col for col in ingredient_info if col in df.columns])
    
    # 3. 메인 이미지
    ordered_columns.extend([col for col in main_image if col in df.columns])
    
    # 4. 기타 컬럼들
    ordered_columns.extend(other_columns)
    
    # 5. 레시피 제작 순서와 이미지를 번호순으로 교대로 배치 (마지막에 배치)
    for i in range(1, 21):
        manual_col = f'MANUAL{i:02d}'
        img_col = f'MANUAL_IMG{i:02d}'
        
        if manual_col in df.columns:
            ordered_columns.append(manual_col)
        if img_col in df.columns:
            ordered_columns.append(img_col)
    
    return df[ordered_columns]

# DataFrame 컬럼 재정렬 실행
if 'df' in locals() and not df.empty:
    print("컬럼 재정렬 전:")
    print("현재 컬럼 순서:", list(df.columns)[:10], "... (총", len(df.columns), "개)")
    
    df_ordered = reorder_recipe_columns(df)
    
    print("\\n컬럼 재정렬 후:")
    print("새로운 컬럼 순서:")
    for i, col in enumerate(df_ordered.columns, 1):
        print(f"{i:2d}. {col}")
    
    # 레시피 제작 순서 컬럼들만 따로 확인
    manual_cols = [col for col in df_ordered.columns if col.startswith('MANUAL') and not col.startswith('MANUAL_IMG')]
    image_cols = [col for col in df_ordered.columns if col.startswith('MANUAL_IMG')]
    other_cols = [col for col in df_ordered.columns if col in ['HASH_TAG', 'RCP_NA_TIP']]
    
    print(f"\\n기타 컬럼 (4번 위치): {len(other_cols)}개")
    print(other_cols)
    
    print(f"\\n레시피 제작 순서 컬럼 (5번 위치): {len(manual_cols)}개")
    print(manual_cols)
    
    print(f"\\n레시피 이미지 컬럼 (5번 위치): {len(image_cols)}개") 
    print(image_cols)
    
    # 원본 df를 재정렬된 것으로 교체
    df = df_ordered
    print(f"\\nDataFrame 컬럼 순서가 재정렬되었습니다!")
    print("순서: 1.기본정보 → 2.재료정보 → 3.메인이미지 → 4.기타컬럼 → 5.제작순서&이미지")
    
else:
    print("DataFrame이 없습니다. 먼저 데이터를 수집해주세요.")

컬럼 재정렬 전:
현재 컬럼 순서: ['RCP_PARTS_DTLS', 'RCP_WAY2', 'MANUAL_IMG20', 'MANUAL20', 'RCP_SEQ', 'INFO_NA', 'INFO_WGT', 'INFO_PRO', 'MANUAL_IMG13', 'MANUAL_IMG14'] ... (총 55 개)
\n컬럼 재정렬 후:
새로운 컬럼 순서:
 1. RCP_SEQ
 2. RCP_NM
 3. RCP_WAY2
 4. RCP_PAT2
 5. INFO_WGT
 6. INFO_ENG
 7. INFO_CAR
 8. INFO_PRO
 9. INFO_FAT
10. INFO_NA
11. RCP_PARTS_DTLS
12. ATT_FILE_NO_MAIN
13. ATT_FILE_NO_MK
14. HASH_TAG
15. RCP_NA_TIP
16. MANUAL01
17. MANUAL_IMG01
18. MANUAL02
19. MANUAL_IMG02
20. MANUAL03
21. MANUAL_IMG03
22. MANUAL04
23. MANUAL_IMG04
24. MANUAL05
25. MANUAL_IMG05
26. MANUAL06
27. MANUAL_IMG06
28. MANUAL07
29. MANUAL_IMG07
30. MANUAL08
31. MANUAL_IMG08
32. MANUAL09
33. MANUAL_IMG09
34. MANUAL10
35. MANUAL_IMG10
36. MANUAL11
37. MANUAL_IMG11
38. MANUAL12
39. MANUAL_IMG12
40. MANUAL13
41. MANUAL_IMG13
42. MANUAL14
43. MANUAL_IMG14
44. MANUAL15
45. MANUAL_IMG15
46. MANUAL16
47. MANUAL_IMG16
48. MANUAL17
49. MANUAL_IMG17
50. MANUAL18
51. MANUAL_IMG18
52. MANUAL19
53. MANUAL_IMG19
54. MANUAL20
55. MANUAL_

In [37]:
# 특정 레시피의 제작 순서와 이미지를 순서대로 확인하는 함수
def show_recipe_steps(df, recipe_index=0):
    """
    특정 레시피의 제작 순서와 이미지 정보를 순서대로 출력
    
    Args:
        df: 레시피 DataFrame
        recipe_index: 확인할 레시피의 인덱스 (기본값: 0)
    """
    if df.empty or recipe_index >= len(df):
        print("유효하지 않은 레시피 인덱스입니다.")
        return
    
    recipe = df.iloc[recipe_index]
    
    print(f"레시피명: {recipe.get('RCP_NM', 'N/A')}")
    print(f"조리방법: {recipe.get('RCP_WAY2', 'N/A')}")
    print(f"요리종류: {recipe.get('RCP_PAT2', 'N/A')}")
    print("=" * 60)
    
    # 재료 정보
    if 'RCP_PARTS_DTLS' in recipe and pd.notna(recipe['RCP_PARTS_DTLS']):
        print(f"재료: {recipe['RCP_PARTS_DTLS']}")
        print("-" * 60)
    
    # 메인 이미지
    if 'ATT_FILE_NO_MAIN' in recipe and pd.notna(recipe['ATT_FILE_NO_MAIN']):
        print(f"메인 이미지: {recipe['ATT_FILE_NO_MAIN']}")
        print("-" * 60)
    
    # 제작 순서와 이미지
    print("제작 순서:")
    step_count = 0
    
    for i in range(1, 21):
        manual_col = f'MANUAL{i:02d}'
        img_col = f'MANUAL_IMG{i:02d}'
        
        manual_text = recipe.get(manual_col, '')
        img_url = recipe.get(img_col, '')
        
        # 제작 순서가 있는 경우만 출력
        if pd.notna(manual_text) and manual_text.strip():
            step_count += 1
            print(f"\\n{step_count}단계: {manual_text}")
            
            if pd.notna(img_url) and img_url.strip():
                print(f"   이미지: {img_url}")
    
    if step_count == 0:
        print("제작 순서 정보가 없습니다.")

# 사용 예시
if 'df' in locals() and not df.empty:
    print("첫 번째 레시피의 제작 순서:")
    print("=" * 80)
    show_recipe_steps(df, 0)
    
    print("\\n\\n두 번째 레시피의 제작 순서:")
    print("=" * 80)
    show_recipe_steps(df, 1)
else:
    print("DataFrame이 없습니다. 먼저 데이터를 수집해주세요.")

첫 번째 레시피의 제작 순서:
레시피명: 새우 두부 계란찜
조리방법: 찌기
요리종류: 반찬
재료: 새우두부계란찜
연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)
고명
시금치 10g(3줄기)
------------------------------------------------------------
메인 이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00028_2.png
------------------------------------------------------------
제작 순서:
\n1단계: 1. 손질된 새우를 끓는 물에 데쳐 건진다.a
   이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_1.png
\n2단계: 2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)를 함께 섞어 그릇에 담는다.b
   이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_2.png
\n3단계: 3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분 정도 찐다.c
   이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_3.png
\n\n두 번째 레시피의 제작 순서:
레시피명: 부추 콩가루 찜
조리방법: 찌기
요리종류: 반찬
재료: [1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)
·양념장 : 저염간장 3g(2/3작은술), 다진 대파 5g(1작은술), 다진 마늘 2g(1/2쪽), 고춧가루 2g(1/3작은술), 요리당 2g(1/3작은술), 참기름 2g(1/3작은술), 참깨 약간
---------------------------------------------------------

In [38]:
# 현재 DataFrame의 빈 값 상태 확인
if 'df' in locals() and not df.empty:
    print("현재 DataFrame의 빈 값 상태:")
    print("=" * 60)
    
    # 빈 문자열이나 공백 값들 확인
    empty_counts = {}
    for col in df.columns:
        # 빈 문자열, 공백만 있는 값, None 등 확인
        empty_str = (df[col] == '').sum()
        whitespace_only = (df[col].str.strip() == '').sum() if df[col].dtype == 'object' else 0
        null_values = df[col].isnull().sum()
        
        empty_counts[col] = {
            'empty_string': empty_str,
            'whitespace_only': whitespace_only,
            'null_values': null_values
        }
    
    # 비어있는 값이 많은 컬럼들 확인
    print("빈 값이 많은 컬럼들 (상위 10개):")
    empty_summary = []
    for col, counts in empty_counts.items():
        total_empty = counts['empty_string'] + counts['whitespace_only'] + counts['null_values']
        if total_empty > 0:
            empty_summary.append((col, total_empty, counts))
    
    empty_summary.sort(key=lambda x: x[1], reverse=True)
    
    for col, total_empty, counts in empty_summary[:10]:
        print(f"{col:20s}: 총 {total_empty:4d}개 빈값 (빈문자열: {counts['empty_string']}, 공백: {counts['whitespace_only']}, null: {counts['null_values']})")
    
    # 샘플 데이터 확인
    print("\n샘플 데이터에서 빈 값 확인:")
    print("-" * 40)
    sample_cols = ['MANUAL10', 'MANUAL_IMG10', 'MANUAL15', 'MANUAL_IMG15']
    for col in sample_cols:
        if col in df.columns:
            unique_vals = df[col].unique()[:5]  # 처음 5개 고유값
            print(f"{col}: {unique_vals}")
            
else:
    print("DataFrame이 없습니다.")

현재 DataFrame의 빈 값 상태:
빈 값이 많은 컬럼들 (상위 10개):
MANUAL07            : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL_IMG07        : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL08            : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL_IMG08        : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL09            : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL_IMG09        : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL10            : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL_IMG10        : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL11            : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)
MANUAL_IMG11        : 총 2272개 빈값 (빈문자열: 1136, 공백: 1136, null: 0)

샘플 데이터에서 빈 값 확인:
----------------------------------------
MANUAL10: ['']
MANUAL_IMG10: ['']
MANUAL15: ['']
MANUAL_IMG15: ['']


In [39]:
# 빈 값들을 null로 치환하는 함수
import numpy as np

def clean_empty_values(df):
    """
    DataFrame의 빈 문자열, 공백만 있는 값들을 null(NaN)로 치환
    """
    df_cleaned = df.copy()
    
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == 'object':  # 문자열 컬럼만 처리
            # 빈 문자열을 NaN으로 치환
            df_cleaned[col] = df_cleaned[col].replace('', np.nan)
            
            # 공백만 있는 문자열을 NaN으로 치환
            df_cleaned[col] = df_cleaned[col].replace(r'^\s*$', np.nan, regex=True)
            
            # 'None', 'null', 'N/A' 등의 문자열도 NaN으로 치환 (필요시)
            df_cleaned[col] = df_cleaned[col].replace(['None', 'null', 'N/A', 'nan'], np.nan)
    
    return df_cleaned

# DataFrame 정리 실행
if 'df' in locals() and not df.empty:
    print("DataFrame 정리 전:")
    print(f"전체 데이터 개수: {len(df)}")
    print(f"빈 문자열 개수 (전체): {(df == '').sum().sum()}")
    
    # DataFrame 정리
    df_cleaned = clean_empty_values(df)
    
    print("\\nDataFrame 정리 후:")
    print(f"null 값 개수 (전체): {df_cleaned.isnull().sum().sum()}")
    
    # 컬럼별 null 값 개수 확인 (null이 있는 컬럼만)
    null_counts = df_cleaned.isnull().sum()
    cols_with_nulls = null_counts[null_counts > 0].sort_values(ascending=False)
    
    print(f"\\nnull 값이 있는 컬럼들 (상위 10개):")
    for col, count in cols_with_nulls.head(10).items():
        percentage = (count / len(df_cleaned)) * 100
        print(f"{col:20s}: {count:4d}개 ({percentage:5.1f}%)")
    
    # 원본 df를 정리된 버전으로 교체
    df = df_cleaned
    print(f"\\nDataFrame이 정리되었습니다!")
    
else:
    print("DataFrame이 없습니다.")

DataFrame 정리 전:
전체 데이터 개수: 1136
빈 문자열 개수 (전체): 33956
\nDataFrame 정리 후:
null 값 개수 (전체): 33956
\nnull 값이 있는 컬럼들 (상위 10개):
MANUAL14            : 1136개 (100.0%)
MANUAL_IMG14        : 1136개 (100.0%)
MANUAL_IMG12        : 1136개 (100.0%)
MANUAL12            : 1136개 (100.0%)
MANUAL_IMG11        : 1136개 (100.0%)
MANUAL11            : 1136개 (100.0%)
MANUAL09            : 1136개 (100.0%)
MANUAL_IMG09        : 1136개 (100.0%)
MANUAL10            : 1136개 (100.0%)
MANUAL_IMG10        : 1136개 (100.0%)
\nDataFrame이 정리되었습니다!


C:\Users\user\AppData\Local\Temp\ipykernel_2544\813990881.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned[col] = df_cleaned[col].replace('', np.nan)


In [40]:
# 정리된 DataFrame 정보 확인
if 'df' in locals() and not df.empty:
    print("정리된 DataFrame 정보:")
    print("=" * 80)
    print(df.info())
    
    print("\\n주요 컬럼들의 null 값 비율:")
    print("-" * 50)
    
    # 컬럼 카테고리별로 null 비율 확인
    basic_cols = ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_ENG']
    manual_cols = [f'MANUAL{i:02d}' for i in range(1, 6)]  # 처음 5개만
    img_cols = [f'MANUAL_IMG{i:02d}' for i in range(1, 6)]  # 처음 5개만
    
    for category, cols in [("기본정보", basic_cols), ("제작순서", manual_cols), ("이미지", img_cols)]:
        print(f"\\n{category} 컬럼들:")
        for col in cols:
            if col in df.columns:
                null_count = df[col].isnull().sum()
                null_pct = (null_count / len(df)) * 100
                print(f"  {col:15s}: {null_count:4d}개 null ({null_pct:5.1f}%)")
    
    print(f"\\n전체 요약:")
    print(f"- 총 레코드 수: {len(df):,}개")
    print(f"- 총 컬럼 수: {len(df.columns)}개") 
    print(f"- 전체 null 값: {df.isnull().sum().sum():,}개")
    
else:
    print("DataFrame이 없습니다.")

정리된 DataFrame 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Data columns (total 55 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   RCP_SEQ           1136 non-null   object 
 1   RCP_NM            1136 non-null   object 
 2   RCP_WAY2          1136 non-null   object 
 3   RCP_PAT2          1136 non-null   object 
 4   INFO_WGT          265 non-null    object 
 5   INFO_ENG          1136 non-null   object 
 6   INFO_CAR          1136 non-null   object 
 7   INFO_PRO          1136 non-null   object 
 8   INFO_FAT          1136 non-null   object 
 9   INFO_NA           1136 non-null   object 
 10  RCP_PARTS_DTLS    1133 non-null   object 
 11  ATT_FILE_NO_MAIN  1134 non-null   object 
 12  ATT_FILE_NO_MK    1135 non-null   object 
 13  HASH_TAG          357 non-null    object 
 14  RCP_NA_TIP        1082 non-null   object 
 15  MANUAL01          1133 non-null   object 
 16  MANUAL_IMG01      1133 n

In [41]:
df.to_csv('레시피.csv', index=False)